In [8]:
import json
import os
import cv2
import glob
import numpy as np

# json data 
json_dir = "./anno"
json_paths = glob.glob(os.path.join(json_dir, "*.json")) 
#  label dict

label_dict = {'수각류':0,'용각류':1,}

for json_path in json_paths :

    #  json 읽기
    with open(json_path, 'r' , encoding='utf-8') as f : 
        json_data = json.load(f)
    #  images, annotations
    images_info = json_data['images'] 
    annotationsinfo = json_data['annotations']

    filename = images_info['filename']
    image_id = images_info['id']
    image_width = images_info['width']
    image_height = images_info['height']

    '''
    이미지 크기가 3024 4032 정도 사이즈라 리사이즈 필요
    하지만 이미지 사이즈 조절시엔 포인트 값에 대해서도 스케일 보정이 필요
    1024*768 위에 이미지가 비율이 4:3
    변경하고자 하는 이미지 크기 설정
    '''

    new_width = 1024
    new_height = 768

    for ann_info in annotationsinfo :
        if image_id == ann_info['image_id'] :

            #image_read
            image_path = os.path.join('./images', filename)
            image = cv2.imread(image_path)

            h,w,c = image.shape # 높이 넓이 칼라채널

            #scale_x = new_width / width
            #image scale
            scale_x = new_width / image.shape[1]
            scale_y = new_height / image.shape[0]

            #resize
            resized_image = cv2.resize(image, (new_width, new_height))

            category_name = ann_info['category_name']
            polygons = ann_info['polygon']


            points = []
            for polygon_info in polygons:
                x = polygon_info['x'] #polygon_info[0]
                y = polygon_info['y'] #polygon_info[1]

                resized_x = int(x * scale_x)
                resized_y = int(y * scale_y)

                points.append([resized_x, resized_y])

            cv2.polylines(resized_image,
                          [
                              np.array(points, np.int32).reshape((-1, 1, 2))

                          ],
                          isClosed=True,
                          color=(255, 0, 0),
                          thickness=5)
            x_coords = [point[0] for point in points]
            y_coords = [point[1] for point in points]

            x_min = min(x_coords)
            y_min = min(y_coords)
            x_max = max(x_coords)
            y_max = max(y_coords)

            cv2.rectangle(resized_image, 
                          (x_min, y_min), 
                          (x_max, y_max), 
                          (0, 255, 0), 
                          2)

            '''
            print(f'')
            cv2.imshow('', resized_image)
            key = cv2.waitkey()
            if key == ord('q'):
                exit()
            # exit()
            '''

            center_x = ((x_max + y_min) / (2 * new_width))
            center_y = ((y_max + x_min) / (2 * new_height))
            yolo_w = (x_max - x_min) / new_width
            yolo_h = (y_max - y_min) / new_height

            #print(f'{center_x}, {center_y}, {yolo_w}, {yolo_h}')
            #file_name
            image_name_temp = filename.replace('.jpg', '')
            temp_string = 'C_TP_15_0007351.jpg'
            #temp_string.split('.') # ['C_TP_15_0007351','jpg']
            #C_TP_15_0007351.jpg -> C_TP_15_0007351.txt
            #label_number
            label_number = label_dict[category_name]
            # (label number, center_x, center_y, yolo_w, yolo_h, image_...)
        os.makedirs('./yolo_label_data', exist_ok=True)
        with open(f'./yolo_label_data/{image_name_temp}.txt', 'a') as f :
            f.write(f'{label_number} {center_x} {center_y} {yolo_w} {yolo_h} {image_id} \n')

        
        





           


